In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
# Cargar los 16 conjuntos de datos
dataset1 = pd.read_csv('SUSQ118CLEAN.csv')
dataset2 = pd.read_csv('SUSQ119CLEAN.csv')
dataset3 = pd.read_csv('SUSQ219CLEAN.csv')
dataset4 = pd.read_csv('SUSQ218CLEAN.csv')
dataset5 = pd.read_csv('SUSQ318CLEAN.csv')
dataset6 = pd.read_csv('SUSQ319CLEAN.csv')
dataset7 = pd.read_csv('SUSQ418CLEAN.csv')
dataset8 = pd.read_csv('SUSQ419CLEAN.csv')
dataset9 = pd.read_csv('CUSQ118CLEAN.csv')
dataset10 = pd.read_csv('CUSQ119CLEAN.csv')
dataset11 = pd.read_csv('CUSQ219CLEAN.csv')
dataset12 = pd.read_csv('CUSQ218CLEAN.csv')
dataset13 = pd.read_csv('CUSQ318CLEAN.csv')
dataset14 = pd.read_csv('CUSQ319CLEAN.csv')
dataset15 = pd.read_csv('CUSQ418CLEAN.csv')
dataset16 = pd.read_csv('CUSQ419CLEAN.csv')

# Concatenar los conjuntos de datos
combined_dataset = pd.concat([dataset1, dataset2,dataset3,dataset4,dataset5,dataset6,dataset7,dataset8,dataset9,dataset10,dataset11,dataset12,dataset13,dataset14,dataset15,dataset16])





In [3]:
combined_dataset.shape

(6684406, 9)

In [4]:
# Eliminar columnas no hacen falta
columnas_a_eliminar = ['start_day', 'start_year']
data = combined_dataset.drop(columnas_a_eliminar, axis=1)

In [5]:
data.head()

,tripduration,from_station_name,to_station_name,usertype,gender,start_month,Age_range
0,323.0,Damen Ave & Pierce Ave,Claremont Ave & Hirsch St,Subscriber,Male,1,30-50
1,377.0,Winthrop Ave & Lawrence Ave,Clark St & Winnemac Ave (Temp),Subscriber,Male,1,30-50
2,2904.0,LaSalle St & Washington St,Troy St & North Ave,Subscriber,Male,1,30-50
3,747.0,Rush St & Hubbard St,Larrabee St & Oak St,Subscriber,Male,1,30-50
4,183.0,Blue Island Ave & 18th St,Paulina St & 18th St,Subscriber,Male,1,30-50


In [6]:
data200 = data.loc[data['usertype'] == 'Customer'].copy()
data200.head()

,tripduration,from_station_name,to_station_name,usertype,gender,start_month,Age_range
0,1481.0,Lake Shore Dr & Ohio St,Lake Shore Dr & Ohio St,Customer,Male,1,30-50
1,1361.0,Lake Shore Dr & Ohio St,Lake Shore Dr & Ohio St,Customer,Male,1,30-50
2,3133.0,Mies van der Rohe Way & Chicago Ave,Shedd Aquarium,Customer,Male,1,30-50
3,3158.0,Mies van der Rohe Way & Chicago Ave,Shedd Aquarium,Customer,Male,1,30-50
4,2958.0,Mies van der Rohe Way & Chicago Ave,Shedd Aquarium,Customer,Male,1,30-50


# Modelos Usertype

# Entrenamiento y normalizacion

Hacer grupos para minimizar las variables categoricas

In [7]:
data['to_station_name'].value_counts().index[:20]

Index(['Streeter Dr & Grand Ave', 'Canal St & Adams St',
       'Clinton St & Washington Blvd', 'Clinton St & Madison St',
       'Michigan Ave & Washington St', 'Kingsbury St & Kinzie St',
       'Daley Center Plaza', 'Lake Shore Dr & North Blvd',
       'Canal St & Madison St', 'Millennium Park', 'Franklin St & Monroe St',
       'Theater on the Lake', 'Michigan Ave & Oak St',
       'Lake Shore Dr & Monroe St', 'Clark St & Elm St',
       'LaSalle St & Jackson Blvd', 'Clinton St & Lake St',
       'Dearborn St & Erie St', 'Larrabee St & Kingsbury St',
       'St. Clair St & Erie St'],
      dtype='object')

In [8]:
data['to_station_name'].value_counts().index[20:40]

Index(['Columbus Dr & Randolph St', 'Wabash Ave & Roosevelt Rd',
       'Wells St & Concord Ln', 'Orleans St & Merchandise Mart Plaza',
       'Wabash Ave & Grand Ave', 'Michigan Ave & Lake St',
       'LaSalle St & Illinois St', 'Wells St & Huron St',
       'Fairbanks Ct & Grand Ave', 'Clark St & Armitage Ave',
       'Morgan St & Lake St', 'Dearborn St & Monroe St',
       'Damen Ave & Pierce Ave', 'State St & Kinzie St',
       'Wells St & Hubbard St', 'Ashland Ave & Division St',
       'Franklin St & Jackson Blvd', 'State St & Randolph St',
       'Kingsbury St & Erie St', 'Indiana Ave & Roosevelt Rd'],
      dtype='object')

In [9]:
data['to_station_name'].value_counts().index[40:60]

Index(['Desplaines St & Kinzie St', 'Green St & Madison St',
       'Wilton Ave & Belmont Ave', 'Clark St & Lincoln Ave',
       'McClurg Ct & Illinois St', 'Dearborn Pkwy & Delaware Pl',
       'Shedd Aquarium', 'Sheffield Ave & Fullerton Ave',
       'Broadway & Barry Ave', 'Wacker Dr & Washington St',
       'Clark St & Randolph St', 'Green St & Randolph St',
       'Clark St & Lake St', 'Clark St & Wrightwood Ave',
       'McClurg Ct & Erie St', 'Wilton Ave & Diversey Pkwy',
       'Bissell St & Armitage Ave', 'Clark St & Drummond Pl',
       'Ravenswood Ave & Lawrence Ave', 'Franklin St & Lake St'],
      dtype='object')

In [10]:
data['to_station_name'].value_counts().index[60:80]

Index(['Marshfield Ave & Cortland St', 'Morgan St & Polk St',
       'Wells St & Elm St', 'Milwaukee Ave & Grand Ave',
       'Desplaines St & Jackson Blvd', 'Sheffield Ave & Waveland Ave',
       'Wells St & Evergreen Ave', 'Michigan Ave & Jackson Blvd',
       'Southport Ave & Roscoe St', 'Lakeview Ave & Fullerton Pkwy',
       'Peoria St & Jackson Blvd', 'Sedgwick St & Huron St',
       'Rush St & Hubbard St', 'Sheffield Ave & Wellington Ave',
       'Rush St & Superior St', 'Sedgwick St & North Ave',
       'Mies van der Rohe Way & Chestnut St', 'Aberdeen St & Jackson Blvd',
       'Loomis St & Lexington St', 'Federal St & Polk St'],
      dtype='object')

In [11]:
data['to_station_name'].value_counts().index[60:80]

Index(['Marshfield Ave & Cortland St', 'Morgan St & Polk St',
       'Wells St & Elm St', 'Milwaukee Ave & Grand Ave',
       'Desplaines St & Jackson Blvd', 'Sheffield Ave & Waveland Ave',
       'Wells St & Evergreen Ave', 'Michigan Ave & Jackson Blvd',
       'Southport Ave & Roscoe St', 'Lakeview Ave & Fullerton Pkwy',
       'Peoria St & Jackson Blvd', 'Sedgwick St & Huron St',
       'Rush St & Hubbard St', 'Sheffield Ave & Wellington Ave',
       'Rush St & Superior St', 'Sedgwick St & North Ave',
       'Mies van der Rohe Way & Chestnut St', 'Aberdeen St & Jackson Blvd',
       'Loomis St & Lexington St', 'Federal St & Polk St'],
      dtype='object')

In [12]:
data['to_station_name'].value_counts().index[100:120]

Index(['LaSalle St & Washington St', 'Clark St & Wellington Ave',
       'Broadway & Belmont Ave', 'Wabash Ave & Wacker Pl',
       'Southport Ave & Waveland Ave', 'Halsted St & Roscoe St',
       'Lake Shore Dr & Wellington Ave', 'Wood St & Milwaukee Ave',
       'Dusable Harbor', 'Michigan Ave & Madison St',
       'Franklin St & Chicago Ave', 'Michigan Ave & 8th St',
       'Pine Grove Ave & Waveland Ave', 'Damen Ave & Chicago Ave',
       'Lake Shore Dr & Diversey Pkwy', 'Aberdeen St & Monroe St',
       'Southport Ave & Wellington Ave', 'Halsted St & Dickens Ave',
       'State St & Van Buren St', 'State St & Pearson St'],
      dtype='object')

In [13]:
data['to_station_name'].value_counts().index[120:140]

Index(['Dayton St & North Ave', 'Halsted St & Clybourn Ave (*)',
       'Stetson Ave & South Water St', 'Broadway & Waveland Ave',
       'Wabash Ave & 16th St', 'Stockton Dr & Wrightwood Ave',
       'Montrose Harbor', 'Michigan Ave & 18th St',
       'Kedzie Ave & Milwaukee Ave', 'Field Blvd & South Water St',
       'Clark St & Chicago Ave', 'Michigan Ave & Pearson St',
       'Lincoln Ave & Roscoe St', 'Larrabee St & Oak St',
       'Sheffield Ave & Willow St', 'Milwaukee Ave & Wabansia Ave',
       'Michigan Ave & 14th St', 'Larrabee St & Division St', 'Burnham Harbor',
       'Ogden Ave & Chicago Ave'],
      dtype='object')

In [14]:
grupo_1=['Streeter Dr & Grand Ave', 'Canal St & Adams St',
       'Clinton St & Washington Blvd', 'Clinton St & Madison St',
       'Michigan Ave & Washington St', 'Kingsbury St & Kinzie St',
       'Daley Center Plaza', 'Lake Shore Dr & North Blvd',
       'Canal St & Madison St', 'Millennium Park', 'Franklin St & Monroe St',
       'Theater on the Lake', 'Michigan Ave & Oak St',
       'Lake Shore Dr & Monroe St', 'Clark St & Elm St',
       'LaSalle St & Jackson Blvd', 'Clinton St & Lake St',
       'Dearborn St & Erie St', 'Larrabee St & Kingsbury St',
       'St. Clair St & Erie St']
grupo_2=['Columbus Dr & Randolph St', 'Wabash Ave & Roosevelt Rd',
       'Wells St & Concord Ln', 'Orleans St & Merchandise Mart Plaza',
       'Wabash Ave & Grand Ave', 'Michigan Ave & Lake St',
       'LaSalle St & Illinois St', 'Wells St & Huron St',
       'Fairbanks Ct & Grand Ave', 'Clark St & Armitage Ave',
       'Morgan St & Lake St', 'Dearborn St & Monroe St',
       'Damen Ave & Pierce Ave', 'State St & Kinzie St',
       'Wells St & Hubbard St', 'Ashland Ave & Division St',
       'Franklin St & Jackson Blvd', 'State St & Randolph St',
       'Kingsbury St & Erie St', 'Indiana Ave & Roosevelt Rd']
grupo_3=['Desplaines St & Kinzie St', 'Green St & Madison St',
       'Wilton Ave & Belmont Ave', 'Clark St & Lincoln Ave',
       'McClurg Ct & Illinois St', 'Dearborn Pkwy & Delaware Pl',
       'Shedd Aquarium', 'Sheffield Ave & Fullerton Ave',
       'Broadway & Barry Ave', 'Wacker Dr & Washington St',
       'Clark St & Randolph St', 'Green St & Randolph St',
       'Clark St & Lake St', 'Clark St & Wrightwood Ave',
       'McClurg Ct & Erie St', 'Wilton Ave & Diversey Pkwy',
       'Bissell St & Armitage Ave', 'Clark St & Drummond Pl',
       'Ravenswood Ave & Lawrence Ave', 'Franklin St & Lake St']
grupo_4=['Marshfield Ave & Cortland St', 'Morgan St & Polk St',
       'Wells St & Elm St', 'Milwaukee Ave & Grand Ave',
       'Desplaines St & Jackson Blvd', 'Sheffield Ave & Waveland Ave',
       'Wells St & Evergreen Ave', 'Michigan Ave & Jackson Blvd',
       'Southport Ave & Roscoe St', 'Lakeview Ave & Fullerton Pkwy',
       'Peoria St & Jackson Blvd', 'Sedgwick St & Huron St',
       'Rush St & Hubbard St', 'Sheffield Ave & Wellington Ave',
       'Rush St & Superior St', 'Sedgwick St & North Ave',
       'Mies van der Rohe Way & Chestnut St', 'Aberdeen St & Jackson Blvd',
       'Loomis St & Lexington St', 'Federal St & Polk St']
grupo_5=['Mies van der Rohe Way & Chicago Ave', 'Rush St & Cedar St',
       'Adler Planetarium', 'University Ave & 57th St',
       'Sheridan Rd & Irving Park Rd', 'Sheffield Ave & Webster Ave',
       'Wabash Ave & 9th St', 'Larrabee St & Webster Ave',
       'Cityfront Plaza Dr & Pioneer Ct', 'Lake Shore Dr & Ohio St',
       'Lincoln Ave & Fullerton Ave', 'Lake Shore Dr & Belmont Ave',
       'Sheffield Ave & Kingsbury St', 'Canal St & Jackson Blvd',
       'Wabash Ave & Adams St', 'Dearborn St & Adams St',
       'Sedgwick St & Webster Ave', 'Clark St & Schiller St',
       'Broadway & Cornelia Ave', 'Desplaines St & Randolph St']
grupo_6=['LaSalle St & Washington St', 'Clark St & Wellington Ave',
       'Broadway & Belmont Ave', 'Wabash Ave & Wacker Pl',
       'Southport Ave & Waveland Ave', 'Halsted St & Roscoe St',
       'Lake Shore Dr & Wellington Ave', 'Wood St & Milwaukee Ave',
       'Dusable Harbor', 'Michigan Ave & Madison St',
       'Franklin St & Chicago Ave', 'Michigan Ave & 8th St',
       'Pine Grove Ave & Waveland Ave', 'Damen Ave & Chicago Ave',
       'Lake Shore Dr & Diversey Pkwy', 'Aberdeen St & Monroe St',
       'Southport Ave & Wellington Ave', 'Halsted St & Dickens Ave',
       'State St & Van Buren St', 'State St & Pearson St']
grupo_7=['Dayton St & North Ave', 'Halsted St & Clybourn Ave (*)',
       'Stetson Ave & South Water St', 'Broadway & Waveland Ave',
       'Wabash Ave & 16th St', 'Stockton Dr & Wrightwood Ave',
       'Montrose Harbor', 'Michigan Ave & 18th St',
       'Kedzie Ave & Milwaukee Ave', 'Field Blvd & South Water St',
       'Clark St & Chicago Ave', 'Michigan Ave & Pearson St',
       'Lincoln Ave & Roscoe St', 'Larrabee St & Oak St',
       'Sheffield Ave & Willow St', 'Milwaukee Ave & Wabansia Ave',
       'Michigan Ave & 14th St', 'Larrabee St & Division St', 'Burnham Harbor',
       'Ogden Ave & Chicago Ave']

In [15]:
# Crear la nueva columna utilizando np.where()
data['to_station'] = np.where(data['to_station_name'].isin(grupo_1), 'grupo_1',
                         np.where(data['to_station_name'].isin(grupo_2), 'grupo_2',
                                  np.where(data['to_station_name'].isin(grupo_3), 'grupo_3',
                                           np.where(data['to_station_name'].isin(grupo_4), 'grupo_4',
                                                    np.where(data['to_station_name'].isin(grupo_5), 'grupo_5',
                                                             np.where(data['to_station_name'].isin(grupo_6), 'grupo_6',
                                                                      np.where(data['to_station_name'].isin(grupo_7), 'grupo_7', 'otro')
                                                                     )
                                                            )
                                                   )
                                          )
                                 )
                        )


In [16]:
data['from_station_name'].value_counts().index[:20]

Index(['Canal St & Adams St', 'Streeter Dr & Grand Ave',
       'Clinton St & Madison St', 'Clinton St & Washington Blvd',
       'Michigan Ave & Washington St', 'Columbus Dr & Randolph St',
       'Lake Shore Dr & Monroe St', 'Kingsbury St & Kinzie St',
       'Franklin St & Monroe St', 'Daley Center Plaza',
       'Canal St & Madison St', 'Lake Shore Dr & North Blvd',
       'Millennium Park', 'Orleans St & Merchandise Mart Plaza',
       'Theater on the Lake', 'LaSalle St & Jackson Blvd',
       'Michigan Ave & Oak St', 'Clark St & Elm St', 'Clinton St & Lake St',
       'Dearborn St & Erie St'],
      dtype='object')

In [17]:
data['from_station_name'].value_counts().index[20:40]

Index(['Larrabee St & Kingsbury St', 'Wells St & Huron St',
       'Wells St & Concord Ln', 'Michigan Ave & Lake St',
       'Wabash Ave & Grand Ave', 'Desplaines St & Kinzie St', 'Shedd Aquarium',
       'Dearborn St & Monroe St', 'Kingsbury St & Erie St',
       'Wabash Ave & Roosevelt Rd', 'Fairbanks Ct & Grand Ave',
       'St. Clair St & Erie St', 'Franklin St & Jackson Blvd',
       'State St & Randolph St', 'Clark St & Armitage Ave',
       'Wells St & Hubbard St', 'LaSalle St & Illinois St',
       'Indiana Ave & Roosevelt Rd', 'Damen Ave & Pierce Ave',
       'Ashland Ave & Division St'],
      dtype='object')

In [18]:
data['from_station_name'].value_counts().index[40:60]

Index(['State St & Kinzie St', 'Wilton Ave & Belmont Ave',
       'McClurg Ct & Illinois St', 'Wacker Dr & Washington St',
       'Clark St & Lincoln Ave', 'Sheffield Ave & Fullerton Ave',
       'Morgan St & Lake St', 'Green St & Madison St',
       'Dearborn Pkwy & Delaware Pl', 'Clark St & Lake St',
       'Franklin St & Lake St', 'Broadway & Barry Ave',
       'Clark St & Randolph St', 'McClurg Ct & Erie St',
       'Clark St & Wrightwood Ave', 'Marshfield Ave & Cortland St',
       'Ravenswood Ave & Lawrence Ave', 'Milwaukee Ave & Grand Ave',
       'Wilton Ave & Diversey Pkwy', 'Morgan St & Polk St'],
      dtype='object')

In [19]:
data['from_station_name'].value_counts().index[60:80]

Index(['Wells St & Elm St', 'Desplaines St & Jackson Blvd',
       'Green St & Randolph St', 'Peoria St & Jackson Blvd',
       'Bissell St & Armitage Ave', 'Wells St & Evergreen Ave',
       'Sedgwick St & Huron St', 'Michigan Ave & Jackson Blvd',
       'Cityfront Plaza Dr & Pioneer Ct', 'Sedgwick St & North Ave',
       'Mies van der Rohe Way & Chestnut St', 'Adler Planetarium',
       'Dusable Harbor', 'Lakeview Ave & Fullerton Pkwy',
       'Loomis St & Lexington St', 'Stetson Ave & South Water St',
       'Rush St & Superior St', 'Clark St & Schiller St',
       'Aberdeen St & Jackson Blvd', 'Clark St & Drummond Pl'],
      dtype='object')

In [20]:
data['from_station_name'].value_counts().index[80:100]

Index(['Wabash Ave & 9th St', 'Federal St & Polk St',
       'LaSalle St & Washington St', 'Sheffield Ave & Wellington Ave',
       'Southport Ave & Roscoe St', 'Mies van der Rohe Way & Chicago Ave',
       'Sheridan Rd & Irving Park Rd', 'University Ave & 57th St',
       'Dearborn St & Adams St', 'Rush St & Cedar St',
       'Lake Shore Dr & Ohio St', 'Larrabee St & Webster Ave',
       'Franklin St & Chicago Ave', 'Sheffield Ave & Webster Ave',
       'Desplaines St & Randolph St', 'Sheffield Ave & Waveland Ave',
       'Sedgwick St & Webster Ave', 'Michigan Ave & 8th St',
       'Canal St & Jackson Blvd', 'State St & Pearson St'],
      dtype='object')

In [21]:
data['from_station_name'].value_counts().index[100:120]

Index(['Rush St & Hubbard St', 'Lake Shore Dr & Belmont Ave',
       'Lincoln Ave & Fullerton Ave', 'Wabash Ave & Adams St',
       'Wabash Ave & Wacker Pl', 'State St & Van Buren St',
       'Clark St & Wellington Ave', 'Broadway & Cornelia Ave',
       'Sheffield Ave & Kingsbury St', 'Halsted St & Roscoe St',
       'Southport Ave & Wellington Ave', 'Michigan Ave & Madison St',
       'Southport Ave & Waveland Ave', 'Pine Grove Ave & Waveland Ave',
       'Clark St & Chicago Ave', 'Montrose Harbor',
       'Lake Shore Dr & Wellington Ave', 'Canal St & Monroe St (*)',
       'Dayton St & North Ave', 'Broadway & Belmont Ave'],
      dtype='object')

In [22]:
data['from_station_name'].value_counts().index[120:140]

Index(['Wood St & Milwaukee Ave', 'Aberdeen St & Monroe St',
       'Larrabee St & Division St', 'Lake Shore Dr & Diversey Pkwy',
       'Halsted St & Clybourn Ave (*)', 'Halsted St & Dickens Ave',
       'Field Blvd & South Water St', 'Broadway & Waveland Ave',
       'Ogden Ave & Chicago Ave', 'Damen Ave & Chicago Ave',
       'Stockton Dr & Wrightwood Ave', 'Larrabee St & Oak St',
       'Halsted St & Wrightwood Ave', 'Michigan Ave & Pearson St',
       'Franklin St & Quincy St', 'Burnham Harbor',
       'California Ave & Milwaukee Ave', 'Wolcott Ave & Polk St',
       'Milwaukee Ave & Wabansia Ave', 'Sheffield Ave & Willow St'],
      dtype='object')

In [23]:
grupo_1=['Canal St & Adams St', 'Streeter Dr & Grand Ave',
       'Clinton St & Madison St', 'Clinton St & Washington Blvd',
       'Michigan Ave & Washington St', 'Columbus Dr & Randolph St',
       'Lake Shore Dr & Monroe St', 'Kingsbury St & Kinzie St',
       'Franklin St & Monroe St', 'Daley Center Plaza',
       'Canal St & Madison St', 'Lake Shore Dr & North Blvd',
       'Millennium Park', 'Orleans St & Merchandise Mart Plaza',
       'Theater on the Lake', 'LaSalle St & Jackson Blvd',
       'Michigan Ave & Oak St', 'Clark St & Elm St', 'Clinton St & Lake St',
       'Dearborn St & Erie St']
grupo_2=['Larrabee St & Kingsbury St', 'Wells St & Huron St',
       'Wells St & Concord Ln', 'Michigan Ave & Lake St',
       'Wabash Ave & Grand Ave', 'Desplaines St & Kinzie St', 'Shedd Aquarium',
       'Dearborn St & Monroe St', 'Kingsbury St & Erie St',
       'Wabash Ave & Roosevelt Rd', 'Fairbanks Ct & Grand Ave',
       'St. Clair St & Erie St', 'Franklin St & Jackson Blvd',
       'State St & Randolph St', 'Clark St & Armitage Ave',
       'Wells St & Hubbard St', 'LaSalle St & Illinois St',
       'Indiana Ave & Roosevelt Rd', 'Damen Ave & Pierce Ave',
       'Ashland Ave & Division St']
grupo_3=['State St & Kinzie St', 'Wilton Ave & Belmont Ave',
       'McClurg Ct & Illinois St', 'Wacker Dr & Washington St',
       'Clark St & Lincoln Ave', 'Sheffield Ave & Fullerton Ave',
       'Morgan St & Lake St', 'Green St & Madison St',
       'Dearborn Pkwy & Delaware Pl', 'Clark St & Lake St',
       'Franklin St & Lake St', 'Broadway & Barry Ave',
       'Clark St & Randolph St', 'McClurg Ct & Erie St',
       'Clark St & Wrightwood Ave', 'Marshfield Ave & Cortland St',
       'Ravenswood Ave & Lawrence Ave', 'Milwaukee Ave & Grand Ave',
       'Wilton Ave & Diversey Pkwy', 'Morgan St & Polk St']
grupo_4=['Wells St & Elm St', 'Desplaines St & Jackson Blvd',
       'Green St & Randolph St', 'Peoria St & Jackson Blvd',
       'Bissell St & Armitage Ave', 'Wells St & Evergreen Ave',
       'Sedgwick St & Huron St', 'Michigan Ave & Jackson Blvd',
       'Cityfront Plaza Dr & Pioneer Ct', 'Sedgwick St & North Ave',
       'Mies van der Rohe Way & Chestnut St', 'Adler Planetarium',
       'Dusable Harbor', 'Lakeview Ave & Fullerton Pkwy',
       'Loomis St & Lexington St', 'Stetson Ave & South Water St',
       'Rush St & Superior St', 'Clark St & Schiller St',
       'Aberdeen St & Jackson Blvd', 'Clark St & Drummond Pl']
grupo_5=['Wabash Ave & 9th St', 'Federal St & Polk St',
       'LaSalle St & Washington St', 'Sheffield Ave & Wellington Ave',
       'Southport Ave & Roscoe St', 'Mies van der Rohe Way & Chicago Ave',
       'Sheridan Rd & Irving Park Rd', 'University Ave & 57th St',
       'Dearborn St & Adams St', 'Rush St & Cedar St',
       'Lake Shore Dr & Ohio St', 'Larrabee St & Webster Ave',
       'Franklin St & Chicago Ave', 'Sheffield Ave & Webster Ave',
       'Desplaines St & Randolph St', 'Sheffield Ave & Waveland Ave',
       'Sedgwick St & Webster Ave', 'Michigan Ave & 8th St',
       'Canal St & Jackson Blvd', 'State St & Pearson St']
grupo_6=['Rush St & Hubbard St', 'Lake Shore Dr & Belmont Ave',
       'Lincoln Ave & Fullerton Ave', 'Wabash Ave & Adams St',
       'Wabash Ave & Wacker Pl', 'State St & Van Buren St',
       'Clark St & Wellington Ave', 'Broadway & Cornelia Ave',
       'Sheffield Ave & Kingsbury St', 'Halsted St & Roscoe St',
       'Southport Ave & Wellington Ave', 'Michigan Ave & Madison St',
       'Southport Ave & Waveland Ave', 'Pine Grove Ave & Waveland Ave',
       'Clark St & Chicago Ave', 'Montrose Harbor',
       'Lake Shore Dr & Wellington Ave', 'Canal St & Monroe St (*)',
       'Dayton St & North Ave', 'Broadway & Belmont Ave']
grupo_7=['Wood St & Milwaukee Ave', 'Aberdeen St & Monroe St',
       'Larrabee St & Division St', 'Lake Shore Dr & Diversey Pkwy',
       'Halsted St & Clybourn Ave (*)', 'Halsted St & Dickens Ave',
       'Field Blvd & South Water St', 'Broadway & Waveland Ave',
       'Ogden Ave & Chicago Ave', 'Damen Ave & Chicago Ave',
       'Stockton Dr & Wrightwood Ave', 'Larrabee St & Oak St',
       'Halsted St & Wrightwood Ave', 'Michigan Ave & Pearson St',
       'Franklin St & Quincy St', 'Burnham Harbor',
       'California Ave & Milwaukee Ave', 'Wolcott Ave & Polk St',
       'Milwaukee Ave & Wabansia Ave', 'Sheffield Ave & Willow St']

In [24]:
# Crear la nueva columna utilizando np.where()
data['from_station'] = np.where(data['from_station_name'].isin(grupo_1), 'grupo_1',
                         np.where(data['from_station_name'].isin(grupo_2), 'grupo_2',
                                  np.where(data['from_station_name'].isin(grupo_3), 'grupo_3',
                                           np.where(data['from_station_name'].isin(grupo_4), 'grupo_4',
                                                    np.where(data['from_station_name'].isin(grupo_5), 'grupo_5',
                                                             np.where(data['from_station_name'].isin(grupo_6), 'grupo_6',
                                                                      np.where(data['from_station_name'].isin(grupo_7), 'grupo_7', 'otro')
                                                                     )
                                                            )
                                                   )
                                          )
                                 )
                        )


In [25]:
data.head()

,tripduration,from_station_name,to_station_name,usertype,gender,start_month,Age_range,to_station,from_station
0,323.0,Damen Ave & Pierce Ave,Claremont Ave & Hirsch St,Subscriber,Male,1,30-50,otro,grupo_2
1,377.0,Winthrop Ave & Lawrence Ave,Clark St & Winnemac Ave (Temp),Subscriber,Male,1,30-50,otro,otro
2,2904.0,LaSalle St & Washington St,Troy St & North Ave,Subscriber,Male,1,30-50,otro,grupo_5
3,747.0,Rush St & Hubbard St,Larrabee St & Oak St,Subscriber,Male,1,30-50,grupo_7,grupo_6
4,183.0,Blue Island Ave & 18th St,Paulina St & 18th St,Subscriber,Male,1,30-50,otro,otro


In [26]:
# Separar las variables predictoras (X) y la variable objetivo (y)
X = data.drop('usertype', axis=1)
y = data['usertype']

# Definir las columnas categóricas y numéricas
categorical_features = ['from_station', 'to_station', 'gender', 'start_month', 'Age_range']
numeric_features = ['tripduration']

In [27]:
# Crear el preprocesador utilizando ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_features),
        ('scaler', StandardScaler(), numeric_features)
    ])

# Aplicar el preprocesador a los datos
X_preprocessed = preprocessor.fit_transform(X)


In [28]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

In [29]:
y_test.head()

70513     Subscriber
179575    Subscriber
680612    Subscriber
630498    Subscriber
381413    Subscriber
Name: usertype, dtype: object

Regresion Logistica

In [30]:
# Crear y entrenar el modelo de regresión logística
logistic_regression = LogisticRegression(max_iter=2000)
# Entrenar el modelo de regresión logística
logistic_regression.fit(X_train, y_train)

LogisticRegression(max_iter=2000)

In [31]:
# Realizar predicciones en el conjunto de prueba
y_pred = logistic_regression.predict(X_test)

# Calcular las métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Imprimir las métricas de evaluación
print("\nRegresión Logística:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Regresión Logística:
Accuracy: 0.8674782067527276
Precision: 0.8559864791625668
Recall: 0.8674782067527276
F1-score: 0.846522347497678


Arbol de decisión

In [32]:
# Construir y entrenar el modelo DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(max_depth=10)
decision_tree.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=10)

In [33]:
# Realizar predicciones en el conjunto de prueba
y_pred = decision_tree.predict(X_test)

In [34]:
# Calcular las métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Imprimir las métricas de evaluación
print("\nDecision Tree Classifier:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Decision Tree Classifier:
Accuracy: 0.8717283948770348
Precision: 0.862697839925924
Recall: 0.8717283948770348
F1-score: 0.8512346731411812


RANDOM FOREST

In [35]:
from sklearn.ensemble import RandomForestClassifier

# Crear una instancia del clasificador Random Forest
random_forest = RandomForestClassifier(max_depth=10)

# Entrenar el clasificador Random Forest
random_forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=10)

In [36]:
# Realizar predicciones en el conjunto de prueba
y_pred = random_forest.predict(X_test)

In [37]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)

# Imprimir las métricas de evaluación
print("\nRandom Forest Classifier:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)



Random Forest Classifier:
Accuracy: 0.871659578033065
Precision: 0.8630213833423457
Recall: 0.871659578033065
F1-score: 0.8506875102526914


XGBOOST

In [38]:
# Convertir las etiquetas de clase a valores numéricos utilizando LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [39]:
y_train_encoded

array([1, 1, 1, ..., 0, 0, 0])

In [40]:
y_test_encoded

array([1, 1, 1, ..., 1, 1, 0])

In [41]:
# Crear una instancia del clasificador XGBoost
xgb_classifier = XGBClassifier(max_depth=10)

# Entrenar el clasificador XGBoost
xgb_classifier.fit(X_train, y_train_encoded)  


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [42]:
# Realizar predicciones en el conjunto de prueba
y_pred = xgb_classifier.predict(X_test)

In [43]:
y_pred

array([1, 1, 1, ..., 1, 1, 0])

In [44]:
y_test

70513     Subscriber
179575    Subscriber
680612    Subscriber
630498    Subscriber
381413    Subscriber
             ...    
765941    Subscriber
140744    Subscriber
862257    Subscriber
188638    Subscriber
72513       Customer
Name: usertype, Length: 1336882, dtype: object

In [45]:
# Calcular las métricas de evaluación
accuracy = accuracy_score(y_test_encoded, y_pred)
precision = precision_score(y_test_encoded, y_pred, average='weighted')
recall = recall_score(y_test_encoded, y_pred, average='weighted')
f1 = f1_score(y_test_encoded, y_pred, average='weighted')


# Imprimir las métricas de evaluación
print("\nXGBoost Classifier:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


XGBoost Classifier:
Accuracy: 0.8757975647813345
Precision: 0.8653293877988089
Recall: 0.8757975647813345
F1-score: 0.8610714050997124


# QUITAMOS GENDER Y PROBAMOS REGRESION LOGISTICA Y XGBOOST

In [46]:
data2 = data.drop('gender', axis=1)
data2.head()

,tripduration,from_station_name,to_station_name,usertype,start_month,Age_range,to_station,from_station
0,323.0,Damen Ave & Pierce Ave,Claremont Ave & Hirsch St,Subscriber,1,30-50,otro,grupo_2
1,377.0,Winthrop Ave & Lawrence Ave,Clark St & Winnemac Ave (Temp),Subscriber,1,30-50,otro,otro
2,2904.0,LaSalle St & Washington St,Troy St & North Ave,Subscriber,1,30-50,otro,grupo_5
3,747.0,Rush St & Hubbard St,Larrabee St & Oak St,Subscriber,1,30-50,grupo_7,grupo_6
4,183.0,Blue Island Ave & 18th St,Paulina St & 18th St,Subscriber,1,30-50,otro,otro


REGRESION LOGISTICA

In [47]:
# Separar las variables predictoras (X) y la variable objetivo (y)
X2 = data2.drop('usertype', axis=1)
y2 = data2['usertype']

# Definir las columnas categóricas y numéricas
categorical_features = ['from_station', 'to_station', 'start_month', 'Age_range']
numeric_features = ['tripduration']

In [48]:
# Crear el preprocesador utilizando ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_features),
        ('scaler', StandardScaler(), numeric_features)
    ])

# Aplicar el preprocesador a los datos
X_preprocessed2 = preprocessor.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X2_train, X2_test, y2_train, y2_test = train_test_split(X_preprocessed2, y2, test_size=0.2, random_state=42)

Regresion logistica

In [49]:
# Crear y entrenar el modelo de regresión logística
logreg2 = LogisticRegression(max_iter=2000)

# Entrenar el modelo de regresión logística
logreg2.fit(X2_train, y2_train)

# Calcular las predicciones en el conjunto de entrenamiento y prueba
train_predictions5 = logreg2.predict(X2_train)
test_predictions5 = logreg2.predict(X2_test)


In [50]:
# Calcular las métricas de evaluación
accuracy2 = accuracy_score(y2_test, test_predictions5)
precision2 = precision_score(y2_test, test_predictions5, average='weighted')
recall2 = recall_score(y2_test, test_predictions5, average='weighted')
f12 = f1_score(y2_test, test_predictions5, average='weighted')

# Imprimir las métricas de evaluación
print("\nRegresion Logistica:")
print("Accuracy:", accuracy2)
print("Precision:", precision2)
print("Recall:", recall2)
print("F1-score:", f12)


Regresion Logistica:
Accuracy: 0.8642804675356539
Precision: 0.8517518779638592
Recall: 0.8642804675356539
F1-score: 0.8416562469117723


XGBOOST

In [51]:
# Convertir las etiquetas de clase a valores numéricos utilizando LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded2 = label_encoder.fit_transform(y2_train)
y_test_encoded2 = label_encoder.transform(y2_test)

In [52]:
# Crear una instancia del clasificador XGBoost
xgb_classifier2 = XGBClassifier(max_depth=10)

# Entrenar el clasificador XGBoost
xgb_classifier2.fit(X2_train, y_train_encoded2)

# Realizar predicciones en el conjunto de prueba
y_pred2 = xgb_classifier2.predict(X2_test)

In [53]:
# Calcular las métricas de evaluación
accuracy2 = accuracy_score(y_test_encoded2, y_pred2)
precision2 = precision_score(y_test_encoded2, y_pred2, average='weighted')
recall2 = recall_score(y_test_encoded2, y_pred2, average='weighted')
f12 = f1_score(y_test_encoded2, y_pred2, average='weighted')

# Imprimir las métricas de evaluación
print("\nXGBoost Classifier:")
print("Accuracy:", accuracy2)
print("Precision:", precision2)
print("Recall:", recall2)
print("F1-score:", f12)


XGBoost Classifier:
Accuracy: 0.8710649107400653
Precision: 0.8604202774808307
Recall: 0.8710649107400653
F1-score: 0.8521488796705952


# CONCLUSIONES MODELOS